# EfficientNetB0 Model 

In [ ]:
# Core Dependencies (for all models)
import os
import numpy as np
import matplotlib.pyplot as plt

# Deep Learning Dependencies
import tensorflow as tf
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

# Custom Model Dependencies (for EfficientNet)
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization

### Load and Pre-process FER2013 Dataset

In [ ]:
# Define the Paths 
dataset_train = 'FER2013/train'
dataset_test = 'FER2013/test'

In [ ]:
# Define parameters
img_size = (48, 48, 1)
batch_size = 64
input_shape = (img_size[0], img_size[1], 3)  # 3-channel input for EfficientNetB0

## Trial #4: EfficientNetB0 Model 

### Create Data Generators 

In [ ]:
# Define Data Generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.15,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    dataset_train,
    target_size=img_size,
    color_mode='greyscale',
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    dataset_train,
    target_size=img_size,
    color_mode='greyscale',
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    dataset_test,
    target_size=img_size,
    color_mode='greyscale',
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

### Load EfficientNetB0 Base

In [ ]:
# Compute class weights
num_classes = len(train_generator.class_indices)
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights_dict = dict(enumerate(class_weights))

### Define the EfficientNetB0 Model

In [ ]:
# Load EfficientNetB0 model (w/ ImageNet weights)
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)

In [ ]:
# Freeze all layers (initially)
for layer in base_model.layers:
    layer.trainable = False

# Add custom classifier head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
output = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

In [ ]:
# Create the model
model_4 = Model(inputs=base_model.input, outputs=output)

# Define loss function with label smoothing
loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.05) 

# Compile the model (Test 4)
model_4.compile(optimizer=Adam(learning_rate=1e-3),
              loss=loss,
              metrics=['accuracy'])
model_4.summary()

### Train the Model

#### Phase 1: Train Frozen Base

In [ ]:
# Initial training (top layer only)
initial_callbacks = [
    EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5)
]

# Train the model
history = model_4.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30,
    callbacks=initial_callbacks,
    class_weight=class_weights_dict
)

#### Phase 2: Fine-Tune Top Layers

In [ ]:
# Fine-tune the model (unfreeze top EfficientNetB0 layers)
for layer in base_model.layers[-50:]:
    layer.trainable = True

# Compile for fine-tuning (lowered learning rate)
model_4.compile(optimizer=Adam(learning_rate=1e-5),
              loss=loss,
              metrics=['accuracy'])

# fine_tune_callbacks = [
#     tf.keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True),
#     tf.keras.callbacks.ReduceLROnPlateau(patience=3, factor=0.5)
# ]

# Continue training (fine-tune entire model)
history_finetune = model_4.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30,
    callbacks=initial_callbacks,
    class_weight=class_weights_dict
)

### Evaluate the Model

In [ ]:
# Final evaluation (model #4)
test_loss, test_accuracy = model_4.evaluate(test_generator)
print(f"Trial Test #4 Accuracy: {test_accuracy:.4f}")

### Plot the Model on Training Histroy 

In [ ]:
# Plot Accuracy 
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Plot Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.tight_layout()
plt.show()

**Question: Was this model successful?** 

**Answer:**
* Accuracy:  
* Loss: 